# European options

Copyright (&copy;) 2004, 2005, 2006, 2007 StatPro Italia srl

This file is part of QuantLib, a free-software/open-source library for financial quantitative analysts and developers - https://www.quantlib.org/

QuantLib is free software: you can redistribute it and/or modify it under the
terms of the QuantLib license.  You should have received a copy of the
license along with this program; if not, please email
<quantlib-dev@lists.sf.net>. The license is also available online at
<https://www.quantlib.org/license.shtml>.

This program is distributed in the hope that it will be useful, but WITHOUT
ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
FOR A PARTICULAR PURPOSE.  See the license for more details.

In [ ]:
import QuantLib as ql
import pandas as pd

### Global parameters

In [ ]:
todaysDate = ql.Date(15, ql.May, 1998)
ql.Settings.instance().evaluationDate = todaysDate

In [ ]:
interactive = 'get_ipython' in globals()

### Option construction

In [ ]:
exercise = ql.EuropeanExercise(ql.Date(17, ql.May, 1999))
payoff = ql.PlainVanillaPayoff(ql.Option.Call, 8.0)

In [ ]:
option = ql.VanillaOption(payoff, exercise)

### Market data

In [ ]:
underlying = ql.SimpleQuote(7.0)
dividendYield = ql.FlatForward(todaysDate, 0.05, ql.Actual365Fixed())
volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), 0.10, ql.Actual365Fixed())
riskFreeRate = ql.FlatForward(todaysDate, 0.05, ql.Actual365Fixed())

### Processes and models

In [ ]:
process = ql.BlackScholesMertonProcess(
    ql.QuoteHandle(underlying),
    ql.YieldTermStructureHandle(dividendYield),
    ql.YieldTermStructureHandle(riskFreeRate),
    ql.BlackVolTermStructureHandle(volatility),
)

In [ ]:
hestonProcess = ql.HestonProcess(
    ql.YieldTermStructureHandle(riskFreeRate),
    ql.YieldTermStructureHandle(dividendYield),
    ql.QuoteHandle(underlying),
    0.1 * 0.1,
    1.0,
    0.1 * 0.1,
    0.0001,
    0.0,
)
hestonModel = ql.HestonModel(hestonProcess)

### Pricing

We'll collect tuples of method name, option value, estimated error, and discrepancy from the analytic formula.

In [ ]:
results = []

#### Analytic formula

In [ ]:
option.setPricingEngine(ql.AnalyticEuropeanEngine(process))
value = option.NPV()
refValue = value

results.append(('Analytic', value, None, None))

#### Heston semi-analytic formula

In [ ]:
option.setPricingEngine(ql.AnalyticHestonEngine(hestonModel))
value = option.NPV()

results.append(('Heston analytic', value, None, abs(value - refValue)))

#### Heston COS method

In [ ]:
option.setPricingEngine(ql.COSHestonEngine(hestonModel))
value = option.NPV()

results.append(('Heston COS', value, None, abs(value - refValue)))

#### Integral method

In [ ]:
option.setPricingEngine(ql.IntegralEngine(process))
value = option.NPV()

results.append(('Integral', value, None, abs(value - refValue)))

#### Finite-difference method

In [ ]:
timeSteps = 801
gridPoints = 800

In [ ]:
option.setPricingEngine(ql.FdBlackScholesVanillaEngine(process, timeSteps, gridPoints))
value = option.NPV()

results.append(('Finite diff.', value, None, abs(value - refValue)))

#### Binomial method

In [ ]:
timeSteps = 801

In [ ]:
for tree in ["JR", "CRR", "EQP", "Trigeorgis", "Tian", "LR", "Joshi4"]:
    option.setPricingEngine(ql.BinomialVanillaEngine(process, tree, timeSteps))
    value = option.NPV()

    results.append(('Binomial (%s)' % tree, value, None, abs(value - refValue)))

#### Monte Carlo method

In [ ]:
option.setPricingEngine(ql.MCEuropeanEngine(process, "pseudorandom", timeSteps=1,
                                            requiredTolerance=0.02, seed=42))
value = option.NPV()

results.append(("Monte Carlo (pseudo-random)", value, option.errorEstimate(), abs(value - refValue)))

In [ ]:
option.setPricingEngine(ql.MCEuropeanEngine(process, "lowdiscrepancy", timeSteps=1,
                                            requiredSamples=32768))
value = option.NPV()

results.append(("Monte Carlo (low-discrepancy)", value, None, abs(value - refValue)))

### Results

In [ ]:
df = pd.DataFrame(results,
                  columns=["Method", "Option value", "Error estimate", "Actual error"])

In [ ]:
df.style.hide_index()

The following displays the results when this is run as a Python script (in which case the cell above is not displayed).

In [ ]:
if not interactive:
    print(df)